In [2]:
import pandas as pd
import geopandas as gpd
import censusdata
import numpy as np
import warnings; warnings.simplefilter('ignore')

In [5]:
#censusdata.search('acs5', 2015, 'concept', 'employment')

In [21]:
#censusdata.printtable(censusdata.censustable('acs5', 2015, 'B17005'))

In [14]:
df = censusdata.download('acs5', 2015, censusdata.censusgeo([('state', '25'), ('county', '025'), ('tract', '*')]),
                             ['B17005_001E', 'B17005_002E', 'B17005_013E'])
df = df.reset_index(drop=False)
df.rename(columns = {'index' : 'TRACT', 'B17005_001E' : 'Total', 'B17005_002E' : 'Income below the poverty line', 
                     'B17005_013E' : 'Income at or above the poverty line'}, inplace = True)
for x in range(len(df)):
    df.loc[x, 'TRACT'] = df['TRACT'].unique()[x].geo[2][1]
    
df = df.astype({"TRACT": int})

df

,TRACT,Total,Income below the poverty line,Income at or above the poverty line
0,20303,2686,539,2147
1,40401,2002,249,1753
2,40801,3386,884,2502
3,50101,4279,907,3372
4,50901,3901,574,3327
...,...,...,...,...
199,101101,3042,536,2506
200,130300,3730,10,3720
201,140102,3925,185,3740
202,140300,5389,739,4650


In [17]:
template = pd.read_csv('Precinct_Make_Up_Of_Census_Tracts.csv')
template = template.drop('Unnamed: 0', axis=1)

prct_inc = pd.DataFrame(0, index=np.arange(len(template.columns[2:])),
                        columns=df.columns[1:])

prct_inc.insert(0, 'WARD_PRECI', template.columns[2:])

for i in range(len(prct_inc)):
    prct = prct_inc['WARD_PRECI'].loc[i]
    temp = template[['TRACT', prct]]
    for n in range(len(temp)):
        if temp[prct].loc[n] != 0:
            dec = temp.loc[n, prct]
            filt = df['TRACT'] == temp.loc[n, 'TRACT']
            for c in df.columns[1:]:
                prct_inc[c].iloc[i] += round(df[filt][c] * dec)

In [28]:
neighborhoods = gpd.read_file('Boston_Neighborhoods.shp')

props = pd.read_csv('Neighborhood_Make_Up_Of_Precincts.csv')

neigh_inc_11_15 = pd.DataFrame(0, index=np.arange(len(neighborhoods['Name'])),
                        columns=prct_hisp.columns[1:])
neigh_inc_11_15.insert(0, 'NEIGHBORHOOD', neighborhoods['Name'])

for i in range(len(neigh_inc_11_15)):
    neighb = neigh_inc_11_15['NEIGHBORHOOD'].loc[i]
    temp = props[['WARD_PRECI', neighb]]
    for n in range(len(temp)):
        if temp[neighb].loc[n] != 0:
            dec = temp[neighb].loc[n]
            filt = prct_inc['WARD_PRECI'] == temp['WARD_PRECI'].loc[n]
            for c in prct_inc.columns[1:]:
                try:
                    neigh_inc_11_15[c].iloc[i] += round(prct_inc[filt][c] * dec)
                except ValueError as v:
                    break

for c in neigh_inc_11_15.columns:
    if c != 'NEIGHBORHOOD':
        neigh_inc_11_15[c] = neigh_inc_11_15[c].round().astype(int)
        
cols = list(neigh_inc_11_15.columns)[2:]

for c in cols:
    for i in range(len(neigh_inc_11_15)):
        neigh_inc_11_15.loc[i, c] = "{:.0%}".format(neigh_inc_11_15.loc[i, c]/neigh_inc_11_15.loc[i, 'Total'])
 
neigh_inc_11_15 = neigh_inc_11_15.drop('Total', axis=1)
neigh_inc_11_15

,NEIGHBORHOOD,Income below the poverty line,Income at or above the poverty line
0,Roslindale,12%,88%
1,Jamaica Plain,19%,81%
2,Mission Hill,40%,60%
3,Longwood,39%,61%
4,Bay Village,18%,82%
5,Leather District,30%,70%
6,Chinatown,28%,72%
7,North End,13%,87%
8,Roxbury,31%,69%
9,South End,19%,81%


In [29]:
neigh_inc_11_15.to_csv('Boston_Neighborhood_Estimated_PovertyLine_Demographics_ACS_2011-2015.csv')

In [31]:
df2 = censusdata.download('acs5', 2019, censusdata.censusgeo([('state', '25'), ('county', '025'), ('tract', '*')]),
                             ['B17005_001E', 'B17005_002E', 'B17005_013E'])
df2 = df2.reset_index(drop=False)
df2.rename(columns = {'index' : 'TRACT', 'B17005_001E' : 'Total', 'B17005_002E' : 'Income below the poverty line', 
                     'B17005_013E' : 'Income at or above the poverty line'}, inplace = True)
for x in range(len(df2)):
    df2.loc[x, 'TRACT'] = df2['TRACT'].unique()[x].geo[2][1]
    
df2 = df2.astype({"TRACT": int})

df2

,TRACT,Total,Income below the poverty line,Income at or above the poverty line
0,802,6076,1613,4463
1,10600,3072,439,2633
2,40100,2075,154,1921
3,40300,3585,280,3305
4,50400,2067,231,1836
...,...,...,...,...
199,82100,4762,1308,3454
200,60200,1928,54,1874
201,90400,3456,800,2656
202,120500,2431,421,2010


In [32]:
prct_inc2 = pd.DataFrame(0, index=np.arange(len(template.columns[2:])),
                        columns=df2.columns[1:])

prct_inc2.insert(0, 'WARD_PRECI', template.columns[2:])

for i in range(len(prct_inc2)):
    prct = prct_inc2['WARD_PRECI'].loc[i]
    temp = template[['TRACT', prct]]
    for n in range(len(temp)):
        if temp[prct].loc[n] != 0:
            dec = temp.loc[n, prct]
            filt = df2['TRACT'] == temp.loc[n, 'TRACT']
            for c in df.columns[1:]:
                prct_inc2[c].iloc[i] += round(df2[filt][c] * dec)

In [33]:
neigh_inc_15_19 = pd.DataFrame(0, index=np.arange(len(neighborhoods['Name'])),
                        columns=prct_inc2.columns[1:])
neigh_inc_15_19.insert(0, 'NEIGHBORHOOD', neighborhoods['Name'])

for i in range(len(neigh_inc_15_19)):
    neighb = neigh_inc_15_19['NEIGHBORHOOD'].loc[i]
    temp = props[['WARD_PRECI', neighb]]
    for n in range(len(temp)):
        if temp[neighb].loc[n] != 0:
            dec = temp[neighb].loc[n]
            filt = prct_inc2['WARD_PRECI'] == temp['WARD_PRECI'].loc[n]
            for c in prct_inc2.columns[1:]:
                try:
                    neigh_inc_15_19[c].iloc[i] += round(prct_inc2[filt][c] * dec)
                except ValueError as v:
                    break

for c in neigh_inc_15_19.columns:
    if c != 'NEIGHBORHOOD':
        neigh_inc_15_19[c] = neigh_inc_15_19[c].round().astype(int)
        
cols = list(neigh_inc_15_19.columns)[2:]

for c in cols:
    for i in range(len(neigh_inc_15_19)):
        neigh_inc_15_19.loc[i, c] = "{:.0%}".format(neigh_inc_15_19.loc[i, c]/neigh_inc_15_19.loc[i, 'Total'])

neigh_inc_15_19 = neigh_inc_15_19.drop('Total', axis=1)
neigh_inc_15_19

,NEIGHBORHOOD,Income below the poverty line,Income at or above the poverty line
0,Roslindale,8%,92%
1,Jamaica Plain,14%,86%
2,Mission Hill,36%,64%
3,Longwood,39%,61%
4,Bay Village,16%,84%
5,Leather District,26%,74%
6,Chinatown,25%,75%
7,North End,9%,91%
8,Roxbury,27%,73%
9,South End,19%,81%


In [34]:
neigh_inc_15_19.to_csv('Boston_Neighborhood_Estimated_PovertyLine_Demographics_ACS_2015-2019.csv')